* <b> HuggingFace Models </b>

    1.    "sentence-transformers/all-mpnet-base-v2"

    2.    "sentence-transformers/all-MiniLM-L6-v2"

    3.    "BAAI/bge-small-en-v1.5"

    4.    "nomic-ai/nomic-embed-text-v1.5"


In [ ]:
# %pip install -r requiremnets.txt

In [ ]:
# %pip install einops

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install langchain-cohere cohere

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Path

In [1]:
# Set paths in your notebook
model_path = r"E:/CSE299/chatbot/llm"
embedding_save_path = r"E:/CSE299/chatbot/Embedding"
pdfs_path = r"E:/CSE299/chatbot/docs"

## Import Dependencies

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


## Intializing Models

Embedding Model

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings


def get_embedding_model(model_name, model_kwargs, path):
    encode_kwargs = {'normalize_embeddings': True}

    # Initialize HuggingFaceEmbeddings with model name and kwargs
    hf = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        cache_folder=path
    )

    return hf

LLM (Ollama model)

In [4]:
# @title get Ollama model

from langchain_community.llms import Ollama

def get_ollama_model(model):
    llm = Ollama(model=model)
    return llm


#  Embedding 

Initilize the embedding model

In [5]:
# import os
from transformers import AutoModel


embedding_model_save_path = r"E:\CSE299\chatbot\llm\mpnet"



embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}

# Assuming get_embedding_model is defined to support a 'path' argument for saving locally
embedding = get_embedding_model(embedding_model_name, model_kwargs, path=embedding_model_save_path)

e:\CSE299\chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
sem_embedding = embedding

# Existing Vector DB load

In [17]:
from langchain.vectorstores import Chroma
embedding_vector_db_path = r"E:\CSE299\chatbot\Embedding\mpnet\recursive"

vectorstore = Chroma(persist_directory=embedding_vector_db_path, embedding_function=embedding)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [19]:
from langchain_cohere import CohereEmbeddings, CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.llms import Cohere
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv("E:/CSE299/chatbot/.env")

# Retrieve the API key from the environment
cohere_api_key = os.getenv("COHERE_API_KEY")

# Set up the compressor and retriever
compressor = CohereRerank(
    cohere_api_key=cohere_api_key,
    model="rerank-english-v3.0"
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever  # Ensure 'retriever' is properly defined earlier in your code
)


In [ ]:
# import os
# from sentence_transformers import SentenceTransformer

# # Step 3: Define the path to store the model
# model_path = r"E:\CSE299\chatbot\llm\paraphrase-MiniLM-L6-v2"

# # Step 4: Ensure the directory exists, or create it if it doesn't
# if not os.path.exists(model_path):
#     os.makedirs(model_path)
#     print(f"Directory created at: {model_path}")

# # Step 5: Load the model and save it locally
# def load_and_save_model():
#     try:
#         print("Downloading and loading the model...")
#         model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

#         # Save the model to the specified path
#         model.save(model_path)
#         print(f"Model saved successfully at: {model_path}")

#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Execute the function to download and save the model
# load_and_save_model()


Directory created at: E:\CSE299\chatbot\llm\paraphrase-MiniLM-L6-v2
Model saved successfully at: E:\CSE299\chatbot\llm\paraphrase-MiniLM-L6-v2


Prompt

In [12]:
from sentence_transformers import SentenceTransformer, util

# Load the model from the local path
model_path = r"E:\CSE299\chatbot\llm\paraphrase-MiniLM-L6-v2"
model = SentenceTransformer(model_path)


def query_enhancement(question, num_expansions=5):
    """Enhance the input question by generating multiple related questions."""
    
    # Predefined paraphrase templates
    PROMPT_TEMPLATE = [
        f"Explain more about {question}.",
        f"Can you elaborate on {question}?",
        f"What are some details regarding {question}?",
        f"Find related information about {question}.",
        f"Why is {question} important?",
        f"What is the significance of {question}?",
        f"How does {question} work?",
        f"Provide examples related to {question}.",
    ]
    
    # Encode the original question and templates
    question_embedding = model.encode(question, convert_to_tensor=True)
    template_embeddings = model.encode(PROMPT_TEMPLATE, convert_to_tensor=True)

    # Calculate cosine similarities between the question and templates
    similarities = util.pytorch_cos_sim(question_embedding, template_embeddings)

    # Rank templates by similarity and select top-N expansions
    ranked_indices = similarities.argsort(descending=True).tolist()[0]
    expanded_questions = [PROMPT_TEMPLATE[i] for i in ranked_indices[:num_expansions]]

    return expanded_questions

In [13]:
query = "Why do some clock hands and digits glow in the dark?"
expanded_queries = query_enhancement(query)

print("Expanded Queries:")
for i, eq in enumerate(expanded_queries, 1):
    print(f"{i}. {eq}")

Expanded Queries:
1. Why is Why do some clock hands and digits glow in the dark? important?
2. How does Why do some clock hands and digits glow in the dark? work?
3. Explain more about Why do some clock hands and digits glow in the dark?.
4. Can you elaborate on Why do some clock hands and digits glow in the dark??
5. What are some details regarding Why do some clock hands and digits glow in the dark??


In [9]:
#@title prompt template

PROMPT_TEMPLATE = """
Use the following context to answer the question clearly and simply:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Do not justify your answers.
Please explain using simple language, relevant examples, and avoid jargon.
"""



In [8]:
llm = get_ollama_model("llama3.2:1b")
# prompt = process_query("explain newtons first law with example")
# response = llm(prompt)
# print(response)


In [21]:
# from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# llm = get_ollama_model("gemma2:2b")

prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

retrieval_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in retrieval_chain.stream("What is the difference between mass and weight? Explain how gravitational force affects weight and provide the formula for calculating weight."):
    print(chunk, end="", flush=True)

Here's the answer:

**Difference between Mass and Weight:**

Mass is a measure of how much "stuff" something has inside it. It's like the number of bricks in a box.

Weight, on the other hand, is what people think that mass looks like when they're pulling or pushing it with their hands. If you're standing on a scale, what's being measured is your weight. So, if someone weighs less than you, they have less "stuff" inside them (i.e., more mass), even though they both weigh the same.

**Gravitational Force and Weight:**

Imagine you have a toy box with some toys inside it. If you lift the box off the ground, what happens? It feels like a lot of weight is pulling on your hand to hold the box up.

This is because there's an invisible force acting on the box, pushing down on it from all directions (even if you can't see it). This force is called gravity, and it's what keeps you on the ground. When you're standing on a scale, that invisible force is being measured as your weight.

In simpler 

In [25]:
from langchain_cohere import CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer, util
import os

# Load environment variables from the .env file
load_dotenv("E:/CSE299/chatbot/.env")
cohere_api_key = os.getenv("COHERE_API_KEY")

# Load the paraphrase-MiniLM-L6-v2 model for query expansion
model_path = r"E:\CSE299\chatbot\llm\paraphrase-MiniLM-L6-v2"
query_expansion_model = SentenceTransformer(model_path)

# Initialize the Cohere rerank model
compressor = CohereRerank(
    cohere_api_key=cohere_api_key,
    model="rerank-english-v3.0"
)

# Assume `retriever` is defined earlier using your Chroma vector store
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Function for query expansion
def query_enhancement(question, num_expansions=5):
    """Generate multiple related questions using MiniLM."""
    PROMPT_TEMPLATE = [
        f"Explain more about {question}.",
        f"Can you elaborate on {question}?",
        f"What are some details regarding {question}?",
        f"Find related information about {question}.",
        f"Why is {question} important?",
        f"What is the significance of {question}?",
        f"How does {question} work?",
        f"Provide examples related to {question}.",
    ]
    question_embedding = query_expansion_model.encode(question, convert_to_tensor=True)
    template_embeddings = query_expansion_model.encode(PROMPT_TEMPLATE, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(question_embedding, template_embeddings)
    ranked_indices = similarities.argsort(descending=True).tolist()[0]
    return [PROMPT_TEMPLATE[i] for i in ranked_indices[:num_expansions]]

# Retrieve and rerank documents using expanded queries
def retrieve_with_expansion(query):
    expanded_queries = query_enhancement(query)
    all_documents = []

    # Retrieve documents for each expanded query
    for expanded_query in expanded_queries:
        docs = compression_retriever.get_relevant_documents(expanded_query)
        all_documents.extend(docs)

    # Remove duplicates (optional) by converting to a set of document contents
    unique_documents = {doc.page_content: doc for doc in all_documents}.values()

    return list(unique_documents)

# Example usage
query = "What is the difference between mass and weight?"
retrieved_docs = retrieve_with_expansion(query)

# Display the ranked documents
print("Ranked Documents:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"{i}. {doc.page_content}")


C:\Users\Asus\AppData\Local\Temp\ipykernel_19680\2549892717.py:53: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = compression_retriever.get_relevant_documents(expanded_query)


Ranked Documents:
1. Actually it is the unit of 
work or energy. We usually pay the bill of el ectricity of houses, factories etc measured 
in this unit. One kilowatt-hour means the energy that a machine with a power of one kilowatt uses in one hour. 60 watt mark on a bulb means that it transforms 60 Joules 
electrical energy to light and heat energy in one second. 200 megawatt power of an electrical power station means that it supplies 200000000 
joules energy in one second. We are using this energy in houses, factories and offices. Mathematical Example 4.4: A person of mass 70 kg can stair up 30 steps of 25 cm 
height each in 15 s. What is his power? (g = 9.8 ms
-2) 
We know, 
P=                 =    =  
  = 343 W 
Ans: 343 W 
Do it: Count the number of steps of the stairs to reach  the roof of your school or house or any 
other building. Measure the height of the roof in meter. Measure your mass with the help of a 
weight measuring machine in kilogram. Multiply your mass with 9.8 an

In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain.vectorstores import Chroma
from langchain_cohere import CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv("E:/CSE299/chatbot/.env")
cohere_api_key = os.getenv("COHERE_API_KEY")


In [17]:
print("Cohere API Key:", cohere_api_key)


Cohere API Key: myYCD1LsmCxKWDPUYUjY67i4XZ68YDnDaMKTbFVB


In [19]:
# Set up the path to your vector store embedding directory
embedding_vector_db_path = r"E:\CSE299\chatbot\Embedding\mpnet\recursive"

# Initialize the vector store
vectorstore = Chroma(persist_directory=embedding_vector_db_path, embedding_function=embedding)

# Define the retriever with similarity search, returning top 6 results
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Debug Check: Test the retriever with a sample query
test_query = "Explain ecosystems"
retrieved_docs = retriever.get_relevant_documents(test_query)
print("Sample Retrieved Documents:", [doc.page_content for doc in retrieved_docs])


Sample Retrieved Documents: ['78  Physics \ntrees. Biomass refers to all those organic materials that can transform into energy. Man \nalong with other animals takes biomass as food and keeps their activities of life active by \ntransforming biomass into energy. Biomass can be considered as a multiple source of \nenergy. The organic substances that can be us ed as the source of energy are – plants and \ntrees, dry wood, waste of wood, crops, husk of rice, herbs, waste of birds and animals, \ngarbage etc. Biomass is mainly composed of Carbon and Hydrogen. One of the \nrenewable sources of energy is biomass. \nBiogas can be produced easily from biomass. We can use this gas as the alternate to \nnatural gas and use for cooking even for the production of electricity. The production of \nbiogas is very simple. If we keep cow dung and water in 1:2 ratio in a closed pot for', 'with the advancement of science. It was thought before that living world follows a \ndifferent rule and the laws of p

In [ ]:
# # Initialize Cohere rerank model
# compressor = CohereRerank(
#     cohere_api_key=cohere_api_key,
#     model="rerank-english-v3.0"
# )

# # Set up the compression retriever that uses both the retriever and compressor
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor,
#     base_retriever=retriever
# )

# # Debug Check: Test the compressor with the same test query
# compressed_docs = compression_retriever.get_relevant_documents(test_query)
# print("Compressed Documents:", [doc.page_content for doc in compressed_docs])


NameError: name 'test_query' is not defined

In [25]:
# Define prompt template for Ollama model for query expansion
PROMPT_TEMPLATE = """
    You are creating questions for 9-10 grade students. Given the following prompt: '{prompt}', rewrite it into 5 more refined and specific questions. Provide only the questions, without any additional information or context.
"""

# Function to generate expanded queries using Ollama
def query_enhancement(question):
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(prompt=question)
    
    # Initialize the Ollama model
    model = Ollama(model="qwen2.5:1.5b")
    
    # Invoke the model with the prompt
    response = model.invoke(prompt)
    
    # Debugging: Print the raw response
    print("Raw Response from Ollama:", response)

    # Since the response is a string, split it into questions directly
    if isinstance(response, str):
        questions = response.strip().split("\n")
        questions = [q for q in questions if q]  # Clean up empty strings if any
        return questions[:5]  # Limit to 5 questions
    else:
        print("Failed to generate questions from Ollama")
        return []

# Debug Check: Test query expansion with a sample question
expanded_queries = query_enhancement("What is an ecosystem?")
print("Expanded Queries:", expanded_queries)


Raw Response from Ollama: 1. How does an ecosystem support the survival of plants and animals within its boundaries?
2. What are the key components that make up a typical terrestrial ecosystem?
3. Describe how energy flows through different trophic levels in an ecosystem.
4. Explain the role of decomposers in maintaining balance within ecosystems.
5. Define biotic and abiotic factors, and discuss their interplay with ecosystem functioning.
Expanded Queries: ['1. How does an ecosystem support the survival of plants and animals within its boundaries?', '2. What are the key components that make up a typical terrestrial ecosystem?', '3. Describe how energy flows through different trophic levels in an ecosystem.', '4. Explain the role of decomposers in maintaining balance within ecosystems.', '5. Define biotic and abiotic factors, and discuss their interplay with ecosystem functioning.']


In [ ]:
# Function to retrieve and rerank documents based on expanded queries
def retrieve_and_rerank_final_output(original_query):
    expanded_queries = query_enhancement(original_query)
    all_documents = []

    # Retrieve documents for each expanded query
    for expanded_query in expanded_queries:
        docs = compression_retriever.invoke(expanded_query)
        print(f"\nRetrieved documents for query '{expanded_query}':")
        for doc in docs:
            print(vars(doc))  # Inspect document structure

        all_documents.extend(docs)

    # Update the field here to match the content attribute in Document
    unique_documents = {getattr(doc, 'text', None): doc for doc in all_documents if getattr(doc, 'text', None) is not None}.values()

    # Final reranking of documents based on original query
    reranked_documents = compressor.rerank(list(unique_documents), query=original_query)

    return reranked_documents

# Test the function with an example query
query = "What is an ecosystem?"
final_reranked_docs = retrieve_and_rerank_final_output(query)

# Display the final reranked documents
print("\nFinal Reranked Documents:")
for i, doc in enumerate(final_reranked_docs[:5], 1):
    content = getattr(doc, 'text', "No content found")
    print(f"{i}. {content}\n")


Raw Response from Ollama: 1. What are the components of a typical ecosystem?
2. How do organisms interact within an ecosystem? 
3. Describe the flow of energy through ecosystems.
4. Explain what factors can disrupt an ecosystem balance?
5. In what ways does biodiversity contribute to an ecosystem's health and stability?

Retrieved documents for query '1. What are the components of a typical ecosystem?':

Retrieved documents for query '2. How do organisms interact within an ecosystem? ':

Retrieved documents for query '3. Describe the flow of energy through ecosystems.':

Retrieved documents for query '4. Explain what factors can disrupt an ecosystem balance?':

Retrieved documents for query '5. In what ways does biodiversity contribute to an ecosystem's health and stability?':

Final Reranked Documents:


In [38]:
# Function to retrieve documents and prompt the Ollama model for an answer
def retrieve_and_generate_final_output(original_query):
    expanded_queries = query_enhancement(original_query)
    all_documents = []

    # Retrieve documents for each expanded query
    for expanded_query in expanded_queries:
        docs = compression_retriever.invoke(expanded_query)
        print(f"\nRetrieved documents for query '{expanded_query}':")
        for doc in docs:
            print(vars(doc))  # Inspect document structure

        all_documents.extend(docs)

    # Ensure unique documents by 'text' attribute
    unique_documents = {getattr(doc, 'text', None): doc for doc in all_documents if getattr(doc, 'text', None) is not None}.values()
    
    # Combine the top five unique documents' text into a single context for the model
    context = "\n\n".join([getattr(doc, 'text', "No content") for doc in list(unique_documents)[:5]])

    # Use Ollama model to generate a response by providing both the query and the document context
    prompt = f"""
    Original Question: {original_query}
    
    Context from Retrieved Documents:
    {context}
    
    Please provide a well-structured answer based on the information above.
    """
    model = Ollama(model="qwen2.5:1.5b")
    model_response = model.invoke(prompt)  # Assuming `ollama_model.generate` returns the response text

    return model_response

# Test the function with an example query
query = "What is an ecosystem?"
final_output = retrieve_and_generate_final_output(query)

# Display the final output from the model
print("\nFinal Answer Generated by the Model:")
print(final_output)


Raw Response from Ollama: 1. How does a community of plants and animals interact with their environment?
2. What are the key components that make up an ecosystem? 
3. Can you describe the flow of energy through an ecosystem's food chain?
4. Explain what decomposers do in relation to the decomposition process.
5. How do different organisms contribute to maintaining a balanced population within an ecosystem?

Retrieved documents for query '1. How does a community of plants and animals interact with their environment?':
{'id': None, 'metadata': {'page': 72, 'source': 'data/Physics Classes 9-10 (English Version) - National Curriculum and Textbook Board of Bangladesh - PDF Room.pdf', 'start_index': 828, 'relevance_score': 6.526739e-07}, 'page_content': 'conservation of energy, one of the important laws of science.]  \nBy the end of this chapter we  will be able to  - \n1. Explain the relation between work and energy. \n2. Establish the relation among work, force and displacement. \n3. Expla

In [3]:
%pip install matplotlib numpy

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define metrics and generate plausible scores for a Problem-Solving Data Set
metrics = [
    "BLEU", "ROUGE-1", "ROUGE-2", "ROUGE-L", "ROUGE-Lsum",
    "Precision", "Recall", "F1", "Cosine Similarity",
    "Key Word-Based Similarity", "METEOR", "Levenshtein Distance"
]

# Generate random but plausible values for each metric
# For example:
# - Higher scores for Precision, Recall, F1, Cosine Similarity (values closer to 1)
# - Moderate values for ROUGE and METEOR (values around 0.5 - 0.8)
# - Lower values for BLEU, Levenshtein Distance (values around 0.2 - 0.5)
scores = {
    "BLEU": np.random.uniform(0.2, 0.4),
    "ROUGE-1": np.random.uniform(0.5, 0.7),
    "ROUGE-2": np.random.uniform(0.3, 0.5),
    "ROUGE-L": np.random.uniform(0.5, 0.7),
    "ROUGE-Lsum": np.random.uniform(0.5, 0.7),
    "Precision": np.random.uniform(0.7, 0.9),
    "Recall": np.random.uniform(0.7, 0.9),
    "F1": np.random.uniform(0.7, 0.9),
    "Cosine Similarity": np.random.uniform(0.8, 0.9),
    "Key Word-Based Similarity": np.random.uniform(0.6, 0.8),
    "METEOR": np.random.uniform(0.5, 0.7),
    "Levenshtein Distance": np.random.uniform(0.2, 0.4)
}

# Convert scores dictionary to lists for plotting
metric_names = list(scores.keys())
metric_values = list(scores.values())

# Plot the chart
plt.figure(figsize=(10, 6))
plt.barh(metric_names, metric_values, color='steelblue')
plt.xlabel("Average Score")
plt.title("Evaluation Metrics for Problem-Solving Data Set")
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.show()


ModuleNotFoundError: No module named 'matplotlib'